# Imports

In [ ]:
import dolfin as dl
import hippylib as hl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from spin.core import problem
from spin.fenics import converter
from spin.hippylib import hessian, laplace, misfit, optimization, prior

sns.set_theme(style="ticks")

# PDE Problem and Data

In [ ]:
mesh = dl.RectangleMesh(dl.Point(-0.75, -0.75), dl.Point(0.75, 0.75), 50, 50)
problem_settings = problem.SPINProblemSettings(
    mesh=mesh,
    pde_type="mean_exit_time",
    inference_type="drift_only",
    log_squared_diffusion=("std::log(std::pow(x[0],2) + std::pow(x[1],2) + 2)", "std::log(1)"),
)
problem_builder = problem.SPINProblemBuilder(problem_settings)
spin_problem = problem_builder.build()

In [ ]:
parameter_coordinates = spin_problem.coordinates_parameters
solution_coordinates = spin_problem.coordinates_variables
true_parameter = converter.create_dolfin_function(
    ("-2*std::pow(x[0]+x[1],3) + 3*x[0] + x[1]", "-x[0]-2*x[1]"),
    spin_problem.function_space_parameters,
)
true_parameter = converter.convert_to_numpy(
    true_parameter.vector(), spin_problem.function_space_parameters
)
true_solution = spin_problem.solve_forward(true_parameter)

In [ ]:
noise_std = 0.01
num_data_points = 100
rng = np.random.default_rng(seed=0)
data_indices = rng.integers(1, true_solution.size - 1, size=num_data_points)
data_locations = solution_coordinates[data_indices, :]
data_values = true_solution[data_indices]
noise = rng.normal(loc=0, scale=noise_std, size=data_values.shape)
data_values = data_values + noise

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), layout="constrained")
contour_plot = axs[0].tricontourf(
    parameter_coordinates[:, 0],
    parameter_coordinates[:, 1],
    true_parameter[0, :],
    levels=50,
    cmap="Blues",
)
fig.colorbar(contour_plot)
axs[0].set_title("True parameter")
axs[0].set_xlabel("x")
axs[0].set_ylabel("y")
axs[0].text(
    0.2, 0.5, r"$b_1(x, y)$", bbox={"facecolor": "white", "boxstyle": "round", "alpha": 0.7}
)
contour_plot = axs[1].tricontourf(
    parameter_coordinates[:, 0],
    parameter_coordinates[:, 1],
    true_parameter[1, :],
    levels=50,
    cmap="Blues",
)
fig.colorbar(contour_plot)
axs[1].set_title("True parameter")
axs[1].set_xlabel("x")
axs[1].set_ylabel("y")
axs[1].text(
    0.2, 0.5, r"$b_2(x, y)$", bbox={"facecolor": "white", "boxstyle": "round", "alpha": 0.7}
)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), layout="constrained")
contour_plot = axs[0].tricontourf(
    solution_coordinates[:, 0],
    solution_coordinates[:, 1],
    true_solution,
    levels=50,
    cmap="Blues",
)
fig.colorbar(contour_plot)
axs[0].set_title("True solution")
axs[0].set_xlabel("x")
axs[0].set_ylabel("y")
axs[0].text(
    0.2, 0.5, r"$\tau(x, y)$", bbox={"facecolor": "white", "boxstyle": "round", "alpha": 0.7}
)
scatter_plot = axs[1].scatter(
    data_locations[:, 0], data_locations[:, 1], c=data_values, cmap="Blues"
)
fig.colorbar(scatter_plot)
axs[1].set_title("Data")
axs[1].set_xlabel("x")
axs[1].set_ylabel("y")
axs[1].text(
    0.4, 0.65, r"$d_\tau(x, y)$", bbox={"facecolor": "gray", "boxstyle": "round", "alpha": 0.5}
)

# Prior

In [ ]:
prior_settings = prior.PriorSettings(
    function_space=spin_problem.function_space_parameters,
    mean=("-x[0] - x[1]", "-x[0] - x[1]"),
    variance=("1", "1"),
    correlation_length=("0.75", "0.75"),
    robin_bc=True,
    robin_bc_const=1.42,
)
prior_builder = prior.BilaplacianVectorPriorBuilder(prior_settings)
spin_prior = prior_builder.build()
prior_variance = spin_prior.compute_variance_with_boundaries(
    method="Randomized", num_eigenvalues_randomized=500
)

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(11, 9), layout="constrained")
contour_plot = axs[0, 0].tricontourf(
    parameter_coordinates[:, 0],
    parameter_coordinates[:, 1],
    spin_prior.mean_array[0, :],
    levels=50,
    cmap="Blues",
)
fig.colorbar(contour_plot)
axs[0, 0].set_title(r"Prior mean $b_1(x,y)$")
axs[0, 0].set_xlabel("x")
axs[0, 0].set_ylabel("y")
contour_plot = axs[0, 1].tricontourf(
    parameter_coordinates[:, 0],
    parameter_coordinates[:, 1],
    prior_variance[0, :],
    levels=50,
    cmap="Blues",
)
fig.colorbar(contour_plot)
axs[0, 1].set_title(r"Prior variance $b_1(x,y)$")
axs[0, 1].set_xlabel("x")
axs[0, 1].set_ylabel("y")
contour_plot = axs[1, 0].tricontourf(
    parameter_coordinates[:, 0],
    parameter_coordinates[:, 1],
    spin_prior.mean_array[0, :],
    levels=50,
    cmap="Blues",
)
fig.colorbar(contour_plot)
axs[1, 0].set_title(r"Prior mean $b_2(x,y)$")
axs[1, 0].set_xlabel("x")
axs[1, 0].set_ylabel("y")
contour_plot = axs[1, 1].tricontourf(
    parameter_coordinates[:, 0],
    parameter_coordinates[:, 1],
    prior_variance[1, :],
    levels=50,
    cmap="Blues",
)
fig.colorbar(contour_plot)
axs[1, 1].set_title(r"Prior variance $b_2(x,y)$")
axs[1, 1].set_xlabel("x")
axs[1, 1].set_ylabel("y")

# Misfit and Inference Model

In [ ]:
misfit_settings = misfit.MisfitSettings(
    function_space=spin_problem.function_space_variables,
    observation_points=data_locations,
    observation_values=data_values,
    noise_variance=np.ones(data_locations.size) * noise_std**2,
)
misfit_builder = misfit.MisfitBuilder(misfit_settings)
spin_misfit = misfit_builder.build()

In [ ]:
inference_model = hl.Model(
    spin_problem.hippylib_variational_problem,
    spin_prior.hippylib_prior,
    spin_misfit.hippylib_misfit,
)

# Optimization

In [ ]:
optimization_settings = optimization.SolverSettings(
    relative_tolerance=1e-8, absolute_tolerance=1e-12, verbose=True
)
initial_guess = spin_prior.mean_array
newton_solver = optimization.NewtonCGSolver(optimization_settings, inference_model)
solver_solution = newton_solver.solve(initial_guess)
print("Termination reason:", solver_solution.termination_reason)

# Low-Rank Hessian

In [ ]:
hessian_settings = hessian.LowRankHessianSettings(
    num_eigenvalues=15,
    num_oversampling=5,
    inference_model=inference_model,
    evaluation_point=[
        solver_solution.forward_solution,
        solver_solution.optimal_parameter,
        solver_solution.adjoint_solution,
    ],
)
eigenvalues, eigenvectors = hessian.compute_low_rank_hessian(hessian_settings)

In [ ]:
index_vector = np.arange(1, eigenvalues.size + 1)
_, ax = plt.subplots(figsize=(4, 4), layout="constrained")
ax.semilogy(index_vector, eigenvalues, marker="o")
ax.set_title("Eigenvalues")
ax.set_xticks((1, 5, 10, 15))
ax.set_xlabel(r"$i$")
ax.set_ylabel(r"$\lambda_i$")

# Laplace Approximation

In [ ]:
laplace_approximation_settings = laplace.LowRankLaplaceApproximationSettings(
    inference_model=inference_model,
    mean=solver_solution.optimal_parameter,
    low_rank_hessian_eigenvalues=eigenvalues,
    low_rank_hessian_eigenvectors=eigenvectors,
)
laplace_approximation = laplace.LowRankLaplaceApproximation(laplace_approximation_settings)
posterior_variance = laplace_approximation.compute_pointwise_variance(
    method="Randomized", num_eigenvalues_randomized=50
)
posterior_predictive = spin_problem.solve_forward(solver_solution.optimal_parameter)

In [ ]:
_, axs = plt.subplots(nrows=1, ncols=2, figsize=(8, 4), layout="constrained")
axs[0].plot(parameter_coordinates, solver_solution.optimal_parameter, label="Mean")
axs[0].fill_between(
    parameter_coordinates.flatten(),
    solver_solution.optimal_parameter - 1.96 * np.sqrt(posterior_variance),
    solver_solution.optimal_parameter + 1.96 * np.sqrt(posterior_variance),
    alpha=0.3,
    label="95% CI",
)
axs[0].set_title("Laplace approximation")
axs[0].set_xlabel(r"$x$")
axs[0].set_ylabel(r"$b(x)$")
axs[0].set_xlim((-1.5, 1.5))
axs[0].legend()

axs[1].plot(solution_coordinates, true_solution)
axs[1].scatter(data_locations, data_values, color="firebrick", label="Data")
axs[1].set_title("Posterior mean predictive")
axs[1].set_xlabel(r"$x$")
axs[1].set_ylabel(r"$\tau(x)$")
axs[1].set_xlim((-1.5, 1.5))
axs[1].legend()